**import packages**

In [ ]:
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import load_model
import random
from tensorflow.keras.optimizers import Adam

**load data and labels**

In [ ]:
d1 = np.load('card_aug_bw1.npy')
ls1 = np.load('labs_aug1.npy')

d2 = np.load('card_aug_bw2.npy')
ls2 = np.load('labs_aug2.npy')

d3 = np.load('card_aug_bw3.npy')
ls3 = np.load('labs_aug3.npy')

d4 = np.load('card_aug_bw4.npy')
ls4 = np.load('labs_aug4.npy')

d5 = np.load('card_aug_bw5.npy')
ls5 = np.load('labs_aug5.npy')

d6 = np.load('card_aug_bw6.npy')
ls6 = np.load('labs_aug6.npy')

d7 = np.load('card_aug_bw7.npy')
ls7 = np.load('labs_aug7.npy')

d8 = np.load('card_aug_bw8.npy')
ls8 = np.load('labs_aug8.npy')

d9 = np.load('card_aug_bw9.npy')
ls9 = np.load('labs_aug9.npy')

d10 = np.load('card_aug_bw10.npy')
ls10 = np.load('labs_aug10.npy')

tot_data = np.concatenate((d1,d2,d3,d4,d5,d6,d7,d8,d9,d10), axis=0)
tot_labs = np.concatenate((ls1,ls2,ls3,ls4,ls5,ls6,ls7,ls8,ls9,ls10), axis=0)

tot_data, tot_labs = shuffle(tot_data, tot_labs)

**preprocess data for deep learning**

In [ ]:
train_data, val_data, train_labels, val_labels = train_test_split(tot_data, tot_labs, test_size=0.1, random_state=42)

train_data = train_data.reshape(train_data.shape[0], 128, 336, 1) #reshape data for deep learning
val_data = val_data.reshape(val_data.shape[0], 128, 336, 1)

num_classes = 5

one_hot_labels_t = np.eye(num_classes)[train_labels - 14] #convert labels to one hot encoding
one_hot_labels_v = np.eye(num_classes)[val_labels - 14]

**show random samples of the dataset**

In [ ]:
idx = random.sample(range(val_data.shape[0]), 40) #select 40 random samples

samples =val_data[idx]
lab_s = one_hot_labels_v[idx]

fig, axs = plt.subplots(5, 8, figsize=(16, 10)) # plot the samples with their labels
for i, ax in enumerate(axs.flatten()):
    ax.imshow(samples[i], cmap='gray')
    label = str(lab_s[i])
    ax.set_title(f"Label: {label}")
    ax.axis('off')
plt.tight_layout()
plt.show()

**define architecture and train model**

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(8, (21,21), activation='relu', input_shape=(128,336,1)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(16, (13,13), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(32, (5,5), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(5, activation='softmax')
])


learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

#Train the model and save the training history
history = model.fit(train_data, one_hot_labels_t, epochs=45, batch_size=30, validation_data=(val_data, one_hot_labels_v))

model.save('my_model6fake.h5') #save trained model

plt.plot(history.history['loss'], label='Training Loss') #plot training loss
plt.plot(history.history['val_loss'], label='Test loss') #plot validation loss
plt.xlabel('Epoch')
plt.ylabel('Loss(Suits)')
plt.legend()
plt.show()


**display misclassified samples**

In [ ]:
pred_labels = model.predict(val_data)
# Convert predictions and true labels to class numbers
pred_classes = np.argmax(pred_labels, axis=1)
true_classes = np.argmax(one_hot_labels_v, axis=1)
# Get misclassified samples
misclassified_indices = np.where(pred_classes != true_classes)[0]
misclassified_images = val_data[misclassified_indices]
misclassified_true_labels = true_classes[misclassified_indices]
misclassified_pred_labels = pred_classes[misclassified_indices]

# Display misclassified samples with labels
for i in range(len(misclassified_indices)):
    plt.imshow(misclassified_images[i], cmap='gray')
    plt.title(f'True label: {misclassified_true_labels[i]+14}, Predicted label: {misclassified_pred_labels[i]+14}')
    plt.show()
